In [27]:
from model import Model

In [43]:
from config import load_config

In [44]:
config = load_config("./config.yaml")

In [39]:
model = Model(config)

In [40]:
model.params_count(config)

({'encoder_params': 9702400,
  'main_params': 45235200,
  'decoder_params': 13505024,
  'ar_decoder_params': 9047040,
  'model_params': 77489664,
  'vocab_params': 32314880,
  'total_params': 109804544},
 {'encoder_a_params': 24256000,
  'main_a_params': 67852800,
  'decoder_a_params': 60772608,
  'ar_decoder_a_params': 36188160,
  'model_a_params': 189069568,
  'vocab_a_params': 45416448,
  'total_a_params': 234486016})

In [26]:
model

Model(
  (attention): Attention(
    (rotary_emb): RotaryEmb()
  )
  (vocab_emb): Embedding(50304, 512)
  (encoder): Encoder(
    (layers): ModuleList(
      (0-2): 3 x Layer(
        (attn): Linear(in_features=512, out_features=1536, bias=False)
        (attn_o): Linear(in_features=512, out_features=512, bias=False)
        (ffn_up): Linear(in_features=512, out_features=2560, bias=False)
        (ffn_down): Linear(in_features=1280, out_features=512, bias=False)
        (attn_norm): RMSNorm((512,), eps=1e-6, elementwise_affine=True)
        (ffn_norm): RMSNorm((512,), eps=1e-6, elementwise_affine=True)
      )
    )
    (latent_w1): Linear(in_features=512, out_features=768, bias=False)
    (latent_w2): Linear(in_features=512, out_features=512, bias=False)
  )
  (main_model): MainModel(
    (layers): ModuleList(
      (0-11): 12 x Layer(
        (attn): Linear(in_features=512, out_features=1536, bias=False)
        (attn_o): Linear(in_features=512, out_features=512, bias=False)
        

In [58]:
import random

In [60]:
eot_idx = 1
pad_idx = 0
b = 8
s = 64
block_size = 8
buffer = torch.randint(1, 16, (512,))

In [61]:
tokens = torch.full((b, s + 1), pad_idx, dtype=torch.long)
eot = torch.argwhere(buffer == eot_idx).squeeze(-1)
eot_num = eot.shape[0]
b_pos_left, b_pos_right, eot_pos = 0, 0, 0

for row in range(b):
    pos = 1
    while pos <= s:
        pad_num = random.randint(0, block_size - 1)
        pos += pad_num
        rest_len = s + 1 - pos

        if eot_pos >= eot_num or (
            eot_pos < eot_num and eot[eot_pos] - b_pos_left + 1 > rest_len
        ):
            b_pos_right = b_pos_left + rest_len
            tokens[row, pos:] = buffer[b_pos_left:b_pos_right]
            pos = s + 1
            b_pos_left = b_pos_right
        else:
            b_pos_right = eot[eot_pos] + 1
            s_len = b_pos_right - b_pos_left
            tokens[row, pos : pos + s_len] = buffer[b_pos_left:b_pos_right]
            end_pad = (block_size - (pos + s_len - 1) % block_size) % block_size
            pos = pos + s_len + end_pad
            b_pos_left = b_pos_right
            eot_pos += 1

In [64]:
tokens[:, 1:].reshape(64, 8)

tensor([[ 5, 10,  3,  4,  7, 15, 11,  1],
        [ 0,  0,  0,  0,  0,  9,  4,  2],
        [ 1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 14,  8,  5,  7, 13],
        [12, 14, 14, 10,  9, 10,  9, 12],
        [ 9,  8,  6, 12, 15,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  5,  5,  2],
        [ 4,  1,  0,  0,  0,  0,  0,  0],
        [ 0,  4,  2,  5,  4, 10, 15, 10],
        [11,  8,  8, 10,  7,  2, 11, 12],
        [ 9, 10,  5, 10,  2,  3, 11, 14],
        [ 3, 11,  2, 10, 12,  7, 14,  1],
        [12,  9,  3,  7,  6,  9, 13,  4],
        [ 3, 14,  3,  5, 10, 11, 15, 13],
        [ 6,  9, 12, 15,  7, 14,  4,  2],
        [10,  9, 14, 14,  5,  9,  7,  1],
        [ 0,  0,  0,  0,  0,  0,  4,  8],
        [ 6,  3, 12,  8,  4,  7, 10, 13],
        [14,  7,  7,  1,  0,  0,  0,  0],
        [ 0,  0,  0,  9,  2, 14, 14,  5],
        [10,  4,  7, 15, 11,  8,  6,  8],
        [ 7,  1,  0,  0,  0,  0,  0,  0],
        [ 0,  0, 12,  7, 10,  5, 13, 13],
        [ 3,  5,  7, 13,  8, 13,  

In [84]:
k = 2
n = 2
e = k + n
include_previous_block = True

In [119]:
doc = torch.tensor([1, 1, 2]).unsqueeze(0)

In [120]:
doc

tensor([[1, 1, 2]])

In [121]:
def mask_mod_2(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    q_r_idx = q_idx % e
    kv_r_idx = kv_idx % e
    q_token = q_r_idx >= k
    kv_token = kv_r_idx >= k
    q_latent = q_r_idx < k
    kv_latent = kv_r_idx < k
    same_block = q_b_idx == kv_b_idx
    causal_mask = q_idx >= kv_idx
    intra_block_token_mask = q_token & same_block & causal_mask
    inter_block_token_mask = False
    if include_previous_block:
        previous_block = q_b_idx == kv_b_idx + 1
        doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
        inter_block_token_mask = q_token & kv_token & previous_block & doc_mask
    intra_block_latent_mask = q_latent & kv_latent & same_block
    return (
        intra_block_token_mask
        | inter_block_token_mask
        | intra_block_latent_mask
    )

In [122]:
result = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result[i, j] = mask_mod_2(0, 0, i, j)

In [123]:
result

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])